# wps_generate_prsn

#### wps_generate_prsn is a process that runs the [generate_prsn](https://github.com/pacificclimate/climate-explorer-data-prep#generate_prsn-generate-snowfall-file) module of PCIC Climate Explorer Data Preparation Tools. Here, the client will try to connect to a remote Thunderbird instance using the url parameter.

In [1]:
from birdy import WPSClient
import requests
import os
from wps_tools.file_handling import copy_http_content
from wps_tools.testing import get_target_url
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile
from wps_tools.output_handling import auto_construct_outputs, txt_to_string, nc_to_dataset

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("thunderbird")
print(f"Using thunderbird on {url}")

Using thunderbird on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thunderbird/wps


In [3]:
thunderbird = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
thunderbird.generate_prsn?

Signature:
thunderbird.generate_prsn(
    prec,
    tasmin=None,
    tasmax=None,
    dry_run=None,
    chunk_size=100,
    output_file='None',
    loglevel='INFO',
)
Docstring:
Generate precipitation as snow file from precipitation and minimum/maximum temperature data

Parameters
----------
prec : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Precipitation file to process
tasmin : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Tasmin file to process
tasmax : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Tasmax file to process
chunk_size : integer
    Number of time slices to be read/written at a time
output_file : string
    Optional custom name of output file
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
dry_run : boolean
    Checks file to ensure compatible with process

Returns
-------
output : ComplexData:mimetype:`applic

#### We can use the docstring to ensure we provide the appropriate parameters.

In [5]:
# Dry-run
pr_file_opendap = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/pr_day_BCCAQv2+ANUSPLIN300_NorESM1-M_historical+rcp26_r1i1p1_19500101-19500107.nc"
tasmin_file_opendap = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/tasmin_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"
tasmax_file_opendap = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/tasmax_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"

dry_output = thunderbird.generate_prsn(pr_file_opendap, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=True)

In [6]:
req = requests.get(dry_output.get()[0])
dependent_varnames = [line.split(":")[-1].strip(" ['']") for line in req.content.decode('utf-8').split("\n") if "dependent_varnames" in line]
assert dependent_varnames == ['pr', 'tasmin', 'tasmax']

In [7]:
# Normal run
output = thunderbird.generate_prsn(pr_file_opendap, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=False, output_file="prsn_test_mixed.nc")

Access the output with **auto_construct_outputs()** or **txt_to_string()** and **nc_to_dataset** from wps_tools.output_handling

In [8]:
# NBVAL_IGNORE_OUTPUT
outputs = auto_construct_outputs(dry_output.get() + output.get())
[type(output) for output in outputs]

[str, netCDF4._netCDF4.Dataset]

In [9]:
# NBVAL_IGNORE_OUTPUT
output_data = output.get()[0]
nc_content = nc_to_dataset(output_data)
print(txt_to_string(dry_output.get()[0]))
print(output_data)

Dry Run
generate_prsn:Dry Run
INFO:dp.generate_prsn:
INFO:dp.generate_prsn:File: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/pr_day_BCCAQv2+ANUSPLIN300_NorESM1-M_historical+rcp26_r1i1p1_19500101-19500107.nc
INFO:dp.generate_prsn:project: CMIP5
INFO:dp.generate_prsn:model: NorESM1-M
INFO:dp.generate_prsn:institute: PCIC
INFO:dp.generate_prsn:experiment: historical,rcp26
INFO:dp.generate_prsn:ensemble_member: r1i1p1
INFO:dp.generate_prsn:dependent_varnames: ['pr']
INFO:dp.generate_prsn:
INFO:dp.generate_prsn:File: https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/tasmin_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc
INFO:dp.generate_prsn:project: CMIP5
INFO:dp.generate_prsn:model: NorESM1-M
INFO:dp.generate_prsn:institute: PCIC
INFO:dp.generate_prsn:experiment: historical,rcp26
INFO:dp.generate_

In [10]:
assert 'prsn' in nc_content.variables.keys()